In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt

from glob import glob
from pathlib import Path

In [ ]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['TF_FORCE_GPU_ALLOW_GROUTH'] = 'true'

In [ ]:
exercise = 'exer3'

In [ ]:
tr_set_group = sorted([x for x in Path(f"./dataset/{exercise}/raw/").glob("*.npy")])
tr_set_group

In [ ]:
if exercise == 'exer1':
    actions = ['thumb','little']
elif exercise == 'exer2':
    actions = ['five', 'four', 'three', 'two', 'one', 'zero']
else:
    actions = ['thumb','paper']

In [ ]:
data = []

for fle in tr_set_group:
    if len(data) == 0:
        data = np.load(fle)
    else:
        data = np.concatenate([data, np.load(fle)], axis=0)

In [ ]:
print(data.shape)
print(data[0][-1])

In [ ]:
x_data = data[:,:-1]
labels = data[:,-1]

print(x_data.shape)
print(labels.shape)
print(labels)

In [ ]:
y_data = to_categorical(labels, num_classes=len(actions))
print(y_data.shape)

In [ ]:
x_data = x_data.astype(np.float32)
y_data = y_data.astype(np.float32)

x_tr,x_val,y_tr,y_val = train_test_split(x_data, y_data, test_size=0.1, shuffle=True, random_state= 3)

print(x_tr.shape, y_tr.shape)
print(x_val.shape, y_val.shape)

In [ ]:
print(x_tr.shape[1:3])
print(x_tr.shape[1:2])

In [ ]:
model = Sequential([
    Dense(64, activation = 'relu', input_shape = x_tr.shape[1:2]),
    Dropout(0.2),
    Dense(32, activation = 'relu'),
    Dropout(0.4),
    Dense(16, activation ='relu'),
    Dense(len(actions), activation = 'softmax')
])

model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['acc']
)

model.summary()

In [ ]:
history = model.fit(
    x_tr, y_tr,
    validation_data = (x_val, y_val),
    epochs = 1000,
    callbacks=[
        ModelCheckpoint(f'models/{exercise}/classifier_acc_raw.h5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto'),
        ModelCheckpoint(f'models/{exercise}/classifier_loss_raw.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='auto'),
        ReduceLROnPlateau(monitor='val_acc', factor=0.05, patience=50, verbose=1, mode='auto'),
        EarlyStopping(monitor='val_loss', min_delta=0, patience=200, mode='auto')
    ]
)

In [ ]:
fig, loss_ax = plt.subplots(figsize=(16,10))
acc_ax = loss_ax.twinx()

loss_ax.plot(history.history['loss'],'y', label='train loss')
loss_ax.plot(history.history['val_loss'], 'r', label='val loss')
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
loss_ax.legend(loc='upper left')

acc_ax.plot(history.history['acc'],'b', label='train acc')
acc_ax.plot(history.history['val_acc'],'g',label='val acc')
acc_ax.set_ylabel('accuracy')
acc_ax.legend(loc='upper left')

plt.show()